In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm

In [17]:
import json
import spacy
from collections import deque

# Cargar intents y evaluaciones para el chat
with open("intents.json", "r", encoding="utf-8") as file:
    intents = json.load(file)

# Cargar datos de celulares
with open("celulares_limpios.json", "r", encoding="utf-8") as file:
    celulares = json.load(file)

# Memoria del chatbot
memoria_chat = deque(maxlen=5)
preferencias_usuario = {}
inicio_recoleccion = False

# Extraer preguntas del flujo desde intents.json
preguntas_flujo = intents["evaluacionesChat"][0]["responses"]
indice_pregunta = 0

# Extraer respuestas positivas y negativas
tags_respuestas = {tag["tag"]: set(tag["responses"]) for tag in intents["evaluacionesChat"] if tag["tag"] in ["respuestas_positivas", "respuestas_negativas"]}

tags_marcas_validas = set()
for tag in intents["evaluacionesChat"]:
    if tag["tag"] == "celulares_por_marca":
        tags_marcas_validas = set(tag["patterns"])
        break

# Cargar modelo de lenguaje para extraer palabras clave
nlp = spacy.load("es_core_news_sm")

def extraer_palabras_clave(texto):
    doc = nlp(texto)
    palabras_clave = [token.lemma_ for token in doc if token.pos_ in ["NOUN", "ADJ", "VERB", "NUM"]]
    return palabras_clave

def recomendar_celular():
    celulares_filtrados = celulares
    for clave, valor in preferencias_usuario.items():
        if clave == "presupuesto":
            celulares_filtrados = [c for c in celulares_filtrados if c["Launched Price (USA)"] <= valor]
        elif clave == "camara":
            celulares_filtrados = [c for c in celulares_filtrados if c["Front Camera (MP)"] >= valor]
        elif clave == "rendimiento":
            celulares_filtrados = [c for c in celulares_filtrados if c["RAM (GB)"] >= valor]
        elif clave == "bateria":
            celulares_filtrados = [c for c in celulares_filtrados if c["Battery Capacity (mAh)"] >= valor]
        elif clave == "marca":
            celulares_filtrados = [c for c in celulares_filtrados if valor.lower() in c["Company Name"].lower()]
    
    return celulares_filtrados[0] if celulares_filtrados else None

def generar_mensaje_recomendacion(celular):
    mensaje = f"Te recomiendo el {celular['Model Name']} de {celular['Company Name']}."

    if "presupuesto" in preferencias_usuario:
        mensaje += f" Su precio de lanzamiento es ${celular['Launched Price (USA)']}."

    if "bateria" in preferencias_usuario and preferencias_usuario["bateria"] > 0:
        mensaje += f" Tiene una batería de {celular['Battery Capacity (mAh)']} mAh para una excelente duración."

    if "camara" in preferencias_usuario and preferencias_usuario["camara"] > 0:
        mensaje += f" Su cámara frontal es de {celular['Front Camera (MP)']} MP y la trasera de {celular['Back Camera (MP)']} MP."

    if "rendimiento" in preferencias_usuario and preferencias_usuario["rendimiento"] > 0:
        mensaje += f" Ofrece un gran rendimiento gracias a su procesador {celular['Processor']} y {celular['RAM (GB)']} GB de RAM."

    mensaje += f" Tiene una pantalla de {celular['Screen Size (inches)']} pulgadas."

    return mensaje

def chatbot():
    global inicio_recoleccion, preferencias_usuario, indice_pregunta
    print("Hola, ¿en qué puedo ayudarte?")
    while True:
        user_input = input("Tú: ").strip()

        if not user_input:
            print("Chatbot: No entendí tu respuesta. Por favor, escribe algo.")
            continue

        if user_input.lower() == "salir":
            print("Chatbot: ¡Hasta luego!")
            break

        if not inicio_recoleccion:
            if "recomendar" in user_input.lower() or "celular" in user_input.lower():
                print("Chatbot: Claro, dime tus preferencias.")
                inicio_recoleccion = True
                indice_pregunta = 0
                preferencias_usuario.clear()
                print(f"Chatbot: {preguntas_flujo[indice_pregunta]['pregunta']}")
            else:
                print("Chatbot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?")
            continue

        clave_pregunta_actual = preguntas_flujo[indice_pregunta]["pregunta"].split("?")[0].strip().lower()

        if clave_pregunta_actual in ["¿te interesa que tenga una batería de larga duración", "¿quieres una buena cámara", "¿necesitas un celular con buen rendimiento"]:
            respuesta = user_input.lower().strip()
            if respuesta in tags_respuestas["respuestas_positivas"]:
                preferencias_usuario[clave_pregunta_actual] = 4000 if "batería" in clave_pregunta_actual else 1
            elif respuesta in tags_respuestas["respuestas_negativas"]:
                preferencias_usuario[clave_pregunta_actual] = 0
            else:
                print("Chatbot: Responde con una opción válida.")
                continue
        elif "presupuesto" in clave_pregunta_actual:
            palabras_clave = extraer_palabras_clave(user_input)
            presupuesto_valido = next((int(palabra) for palabra in palabras_clave if palabra.isdigit()), None)
            if presupuesto_valido is not None:
                preferencias_usuario["presupuesto"] = presupuesto_valido
            else:
                print("Chatbot: Por favor, ingresa un valor numérico para el presupuesto.")
                continue
        elif "marca" in clave_pregunta_actual:
            marca_usuario = user_input.strip().lower()
            if marca_usuario in tags_marcas_validas:
                preferencias_usuario["marca"] = marca_usuario
            else:
                print("Chatbot: Por favor, menciona una marca válida de la lista.")
                continue

        indice_pregunta += 1

        if indice_pregunta < len(preguntas_flujo):
            print(f"Chatbot: {preguntas_flujo[indice_pregunta]['pregunta']}")
        else:
            celular_recomendado = recomendar_celular()
            respuesta = (
                generar_mensaje_recomendacion(celular_recomendado)
                if celular_recomendado
                else "No encontré un celular que cumpla con todas tus preferencias."
            )
            print(f"Chatbot: {respuesta}")
            inicio_recoleccion = False
            preferencias_usuario.clear()
            indice_pregunta = 0

chatbot()


Hola, ¿en qué puedo ayudarte?
Chatbot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?
Chatbot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?
Chatbot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?
Chatbot: Lo siento, no entendí tu pregunta. ¿Puedes reformularla?
Chatbot: ¡Hasta luego!
